In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
from xgboost import plot_importance, plot_tree
import pandas as pd
import numpy as np

## Metrik

In [ ]:

# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    # Nur Fälle berücksichtigen, bei denen y_true nicht 0 ist
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

In [ ]:
# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

## TimeSeries Kreuzvalidierung

...

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
# Zeitreihen-Kreuzvalidierung einrichten
tscv = TimeSeriesSplit(n_splits=5)

## Daten einlesen

...

In [ ]:

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'fourier_sin_365', 'fourier_cos_365',	'days_since_last_holiday',	'days_until_next_holiday'] # lag_1, lag_7 entfernt

# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']

# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['DayOfWeek', 'StoreType', 'StateHoliday','Assortment', 'Store'] # 'Store' entfernt

## Split

Der folgende Code beschreibt die Aufteilung der Daten in einen Trainings- und Testdatensatz zur Vorbereitung für maschinelles Lernen. Zunächst wird der Datensatz anhand der Spalte „Date“ sortiert, um sicherzustellen, dass die zeitliche Reihenfolge der Datenpunkte korrekt ist. Dies ist  notwendig,da es sich um eine Zeitreihenanalyse handelt, bei der die Reihenfolge der Daten für die Genauigkeit der Vorhersage entscheidend ist.

Im nächsten Schritt wird die Zielvariable „Sales“ festgelegt. Sie wird separat als y definiert, da sie die zu prognostizierenden Werte enthält. Gleichzeitig werden die Spalten „Sales“ und „Date“ aus den Features (X) entfernt, da „Sales“ die Zielgröße ist und „Date“ in diesem Kontext keine erklärende Variable darstellt. Dies hilft, das Modell auf die relevanten Merkmale des Datensatzes zu reduzieren.

Zuletzt erfolgt die Aufteilung des Datensatzes in Trainings- und Testdaten. Die ersten 80 % der Daten werden für das Training des Modells verwendet, während die letzten 20 % als Testdaten reserviert werden, um das Modell auf noch nicht gesehenen Daten zu evaluieren. Dieser Ansatz, bei dem die Testdaten aus den letzten Datenpunkten bestehen, ist besonders sinnvoll für zeitbasierte Modelle, da zukünftige Werte auf Grundlage vergangener Ereignisse vorhergesagt werden sollen.

In [ ]:
# Datensatz sortieren, falls nicht bereits geschehen (angenommen, du hast eine Spalte 'Date')
data = data.sort_values('Date')

# ...
data = data[data ['Open']!=0]
data = data[data ['Sales']>0]

# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date', 'Open'], axis=1)  # 'Date' wird entfernt, wenn es keine erklärende Variable ist
y = data['Sales']

# Berechnung der Anzahl der Testdaten (20 % des Datensatzes)
test_size = int(len(data) * 0.2)

# Aufteilen der Daten in Trainings- und Testdaten
X_train, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
y_train, y_test = y.iloc[:-test_size], y.iloc[-test_size:]

## Pre-Processing

Im Preprocessing wird ein ColumnTransformer verwendet, um numerische, bereits encodierte und noch nicht encodierte kategoriale Merkmale eines Datensatzes gezielt zu transformieren. Dieser Schritt ist essenziell, um eine konsistente und effektive Vorbereitung der Daten zu gewährleisten.

Die numerischen Merkmale, die in der Liste numerical_features enthalten sind, umfassen Variablen wie year, month, day, week_of_year, sowie Lag-Features (lag_1, lag_7, lag_30), Fourier-Transformationen zur Erfassung saisonaler Zyklen und Variablen zur Berücksichtigung von Feiertagen. Diese werden mit dem StandardScaler standardisiert, um sicherzustellen, dass alle numerischen Werte auf einen einheitlichen Maßstab gebracht werden.

Bereits One-Hot-encodierte Merkmale (already_encoded_features) wie Open, Promo und promo2 werden ohne weitere Transformation mit 'passthrough' durchgeschleust. Diese Variablen wurden entweder schon in vorherigen Schritten in eine geeignete Form gebracht oder bedürfen keiner weiteren Skalierung oder Encodierung.

Die noch nicht encodierten kategorialen Merkmale, wie DayOfWeek, StoreType, StateHoliday und Assortment, werden mittels OneHotEncoder in binäre Merkmale umgewandelt. Dies gewährleistet, dass jedes Merkmal durch eine Reihe von Spalten dargestellt wird, die angeben, ob eine bestimmte Kategorie vorliegt. Um potenziell unbekannte Kategorien in den Testdaten zu berücksichtigen, wird die Option handle_unknown='ignore' verwendet, wodurch unbekannte Werte ignoriert werden, ohne einen Fehler zu verursachen.

Durch diesen modularen Ansatz werden die numerischen und kategorialen Features des Datensatzes effizient für das Training des Machine-Learning-Modells vorbereitet.

In [ ]:
# Erstelle den Preprocessor für numerische und kategorische Features (ohne Datumsextraktion)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Skalierung für numerische und bereits encodierte Features
        ('enc', 'passthrough', already_encoded_features),  # Bereits encodierte Features durchschleusen (keine weitere Transformation)
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)  # Nur noch nicht encodierte Features encodieren
    ])

## Lag-Features (bitte alle ausprobieren) - nachher bei jedem im eigenen Notebook

In [ ]:
 #pipeline mit preprocessor & model definieren
# Pipeline für die Modellierung
pipeline  = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Schritt 1: Vorverarbeitung
    ('model', ... )  # Schritt 3: Modellierung
])

In [ ]:
# Verbesserte iterative Vorhersagefunktion (mit Berücksichtigung von Lag- und zusätzlichen Features)
def iterative_forecast(pipeline, X_fold_train, y_fold_train, X_fold_val):
    # Trainiere das Modell mit allen Features
    pipeline.named_steps['model'].fit(X_fold_train, y_fold_train)
    
    # Vorhersagen auf Basis der Testdaten
    predictions = []
    last_known_sales = list(y_fold_train[-7:])  # Letzte 30 Tage tatsächlicher Sales-Daten
    
    # Iteriere über die Zeilen von X_fold_val
    for index, row in X_fold_val.iterrows():
        try:
            # Lag-Features für den nächsten Tag berechnen
            row["lag_1"] = last_known_sales[-1]  # Lag-1 ist der zuletzt vorhergesagte Wert
            row["lag_7"] = last_known_sales[-7] if len(last_known_sales) >= 7 else np.nan  # Lag-7 ist 7 Tage zurück
            #row["lag_30"] = last_known_sales[-30] if len(last_known_sales) >= 30 else np.nan  # Lag-30 ist 30 Tage zurück
        except IndexError as e:
            print(f"Error in creating lag features for row {index}: {e}")
            row["lag_1"] = np.nan
            row["lag_7"] = np.nan
            #row["lag_30"] = np.nan
        
        # Wandeln der Zeile in einen DataFrame, um den Preprocessing-Schritt durchzuführen
        row = row.to_frame().T
        
        # Wende das Preprocessing an
        row_preprocessed = pipeline.named_steps['preprocessor'].transform(row)
        
        # Vorhersage für den nächsten Tag
        pred = pipeline.named_steps['model'].predict(row_preprocessed)
        predictions.append(pred[0])  # pred ist ein Array, daher pred[0]
        
        # Update der letzten bekannten Sales-Daten
        last_known_sales.append(pred[0])
    
    return predictions

# Durchführung der Kreuzvalidierung auf den Trainingsdaten
rmspe_scores = []  # Liste zur Speicherung der RMSPE-Scores

for fold, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {fold + 1}")
    
    # Train- und Testdaten für diesen Fold
    X_fold_train = X_train.iloc[train_index]
    X_fold_test = X_train.iloc[test_index]
    y_fold_train = y_train.iloc[train_index]
    y_fold_test = y_train.iloc[test_index]
    
    # Preprocessing auf die Train- und Testdaten anwenden
    X_fold_train_preprocessed = pipeline.named_steps['preprocessor'].fit_transform(X_fold_train)
    X_fold_test_preprocessed = pipeline.named_steps['preprocessor'].transform(X_fold_test)
    
    # Iterative Vorhersage über die Testperiode
    y_pred = iterative_forecast(pipeline, X_fold_train_preprocessed, y_fold_train, X_fold_test)
    
    # RMSPE berechnen und speichern
    rmspe_score = rmspe(y_fold_test, y_pred)
    rmspe_scores.append(rmspe_score)
    
    print(f"RMSPE for Fold {fold + 1}: {rmspe_score}")

# Gesamtergebnisse
print(f"Durchschnittliche RMSPE über alle Folds: {np.mean(rmspe_scores)}")